# Calcul de la distance au point de branchement
Il peut être intéressant de connaitre la distance entre la maison du client et celle du point branchement auquel il doit être rattaché car un certain nombre d'échec de production et de rendez-vous inutiles vienne d'une distance trop importante entre les 2 : le technicien vient sans le matériel adapté ou ne trouve pas le Point de Branchement.

Ce notebook permet de traiter les adresses présentes dans le fichier Excel ETI31 et de calculer leurs coordonnées GPS.

Il faudra également ajouter les adresses des Points de Branchement associés à chacune des adresses pour pouvoir calculer la distance et pour l'intégrer aux notebooks de prévisions d'échecs.

Les étapes de ce notebook :
- Importation des bibliothèques
- Proxy
- Fonction pour la conversion Adresse /GPS et calcul de la distance
- Importation ETI31 et préparation des données
- Coordonnées GPS pour les adresses clients

### Importation des bibliothèques

In [163]:
from geopy.geocoders import Nominatim
from geopy import distance
import requests
import http.client, urllib.parse
import random
import time
import socket
socket.getaddrinfo('localhost', 8888)
import json
import numpy as np
import math
import re
import urllib3
import os
import pickle
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Proxy

### Fonctions pour la conversion Adresse / GPS et calcul de la distance

In [202]:
#Fonction qui prend en entrée une adresse et renvoie la latitude et la longitude associée
def adress_latitude_longitude(adresse):
    api_key = "192349ff78a82f7905ff72c0850de2cd"
    country = "FR"
    url = f"http://api.positionstack.com/v1/forward?access_key={api_key}&query={adresse}&country={country}"
    response = requests.get(url)
    data = response.json()
    
    if data["data"]:
        # Extraire les informations de géocodage
        try :
            latitude = data["data"][0]["latitude"]
            longitude = data["data"][0]["longitude"]
        except:
            print("erreur (data liste) pour :" + adresse)
            latitude = None
            longitude = None
        return latitude,longitude
    else:
        print("erreur (adresse non trouvée) pour :" + adresse)
        return None,None

In [203]:
#Test
adress_latitude_longitude("106 Boulevard Georges Clemenceau 35200 Rennes")

(48.093828, -1.676935)

In [204]:
#Fonction qui calcule la distance entre deux points A et B à partir de leur latitude et longitude
def distance_lat_long(A_lat,A_long,B_lat,B_long):
    A =(A_lat, A_long)
    B =(B_lat , B_long)
    distance = round(GD(A,B).m,2)
    print("La distance est de : " + str(distance) + " mètres.")
    return distance

In [205]:
#Fonction qui calcule la distance à partir de 2 adresses
def distance_2_adresses(adresseA,adresseB):
    A_lat,A_long = adresse_latitude_longitude(adresseA)
    B_lat,B_long = adresse_latitude_longitude(adresseB)
    if A_lat != null and A_long != null and B_lat != null and B_long != null :
        return distance_lat_long(A_lat,A_long,B_lat,B_long)
    else :
        print ("Erreur")
        return 

### Importation des adresses de ETI31 et préparation des données

In [206]:
#Importation avec Pandas
path_eti31 = "csv/ETI31_adresse_client.csv"
df = pd.read_csv(path_eti31,low_memory=False).drop(columns=["CODE INTERVENTION","CODE VOIE"])

In [207]:
df

,No DESIGNATION,CODE RELEVE,DI_ADR_NUMV,DI_ADR_LIBVOIE,DI_ADR_LIBCOMM
0,299568158,RRC,4,LA BRETECHE,MEILLAC
1,297892193,DMS,28,RUE DE KERULVE,LORIENT
2,297897767,DMS,44,RTE PINS,BELZ
3,223370510,DMS,4,RUE DE L OISEAU MARTIN,CHATEAUGIRON
4,299556603,DMS,23,RUE ROBERT DOISNEAU,ACIGNE
...,...,...,...,...,...
5028,296611510,DMS,62,BD WALDECK ROUSSEAU,SAINT BRIEUC
5029,223273821,NaN,NaN,NaN,NaN
5030,223309738,DMS,11,SQ DE TERRE NEUVE,RENNES
5031,298170718,DMS,NaN,MOULIN NEUF,MOTREFF


In [208]:
#Labélisation
label_echec = ["ANC","ANN","ETU","MAJ","ORT","PAD","PBC","REO","RMC","RMF","RRC"]
def colonne_label():
    colonne = []
    for i in df["CODE RELEVE"]:
        if i in label_echec:
            colonne.append(False)
        else :
            colonne.append(True)
    return colonne
df["Reussite"]=colonne_label() 
df = df.drop(columns="CODE RELEVE")

In [209]:
#Nettoyafe des lignes où il manque des données
df = df.dropna(subset=['No DESIGNATION', "DI_ADR_NUMV","DI_ADR_LIBVOIE","DI_ADR_LIBCOMM","Reussite"]).reset_index(drop=True)

In [210]:
#Fonction pour réunir les informations de chaque colonne en une seule colonne adresse
def colonne_adresse():
    new_col = []
    for i in range(0,len(df)):
        new_col.append(str(df.loc[i,"DI_ADR_NUMV"]) + " " + str(df.loc[i,"DI_ADR_LIBVOIE"]) + " " + str(df.loc[i,"DI_ADR_LIBCOMM"]))
    return new_col
df["ADRESSE"]=colonne_adresse()

#On supprimer les colonnes qui sont maintenant inutiles
df = df.drop(columns=["DI_ADR_NUMV","DI_ADR_LIBVOIE","DI_ADR_LIBCOMM"])

In [211]:
#Le dataframe a la fin de la préparation
df

,No DESIGNATION,Reussite,ADRESSE
0,299568158,False,4 LA BRETECHE MEILLAC
1,297892193,True,28 RUE DE KERULVE LORIENT
2,297897767,True,44 RTE PINS BELZ
3,223370510,True,4 RUE DE L OISEAU MARTIN CHATEAUGIRON
4,299556603,True,23 RUE ROBERT DOISNEAU ACIGNE
...,...,...,...
4030,297702145,True,10 LA TOUCHE MORGAN SERENT
4031,299001524,True,3 RUE PERCEVAL CHATEAUGIRON
4032,296611510,True,62 BD WALDECK ROUSSEAU SAINT BRIEUC
4033,223309738,True,11 SQ DE TERRE NEUVE RENNES


### Coordonnées GPS pour les adresses clients

In [216]:
def listeAdresses_listesGPS(listeAdresses):
    liste_latitude = []
    liste_longitude = []
    longueur = len(listeAdresses)
    i = 0
    for adresse in listeAdresses:
        if i%50 == 0: 
            print(str(i) + " sur " + str(longueur))
        i = i+1
        lat_adresse,long_adresse = adress_latitude_longitude(adresse)
        liste_latitude.append(lat_adresse)
        liste_longitude.append(long_adresse)
    return liste_latitude,liste_longitude

On va séparer le dataframe en 40 parties pour éviter de perdre des données inutilement et du temps aussi. Histoire que s'il y a une erreur à la ligne 4000 on perde pas tous les calculs qui ont été fait avant. On aura juste à reconcaténer les dataframe derrière.


In [217]:
len(df)

4035

In [218]:
def creerdf(df,nbr_lignes):
    borne_inf = 0
    borne_sup = nbr_lignes-1
    limite = len(df)
    dfs = []
    ajout = True
    while ajout :
        dfs.append(df.loc[borne_inf:borne_sup])
        
        #Si la dernière ligne ajoutée est la dernière ligne du dataframe
        if borne_sup == limite-1:
            ajout = False
        
        #Si il y a nbr_lignes lignes ou moins à ajouter
        elif borne_sup + nbr_lignes > limite :
            borne_sup = borne_sup+1
            dfs.append(df.loc[borne_sup:limite])
            ajout = False
        
        #Sinon
        else :
            borne_inf=borne_sup+1
            borne_sup = borne_sup + nbr_lignes
    
    return dfs

In [219]:
dfs = creerdf(df,200)

In [220]:
len(dfs)

21

In [223]:
def pickeling_results(liste_df):
    for i in range(17,len(liste_df)):
        print("On fait le " + str(i) + "ème sur " + str(len(liste_df)))
        df_i = liste_df[i]
        df_i["Latitude"],df_i["Longitude"] = listeAdresses_listesGPS(df_i["ADRESSE"])
        name = "pickle/gps/df"+str(i+1) +"_lat_long.pkl"
        df_i.to_pickle(name)        

In [ ]:
pickeling_results(dfs)

On fait le 10ème sur 21
0 sur 200
erreur (data liste) pour :10 R DE KEROCARD SAINT AVE.
erreur (data liste) pour :1E AV GEN DE GAULLE TREGUEUX
50 sur 200
erreur (data liste) pour :18 RUE DES ROSIERS BAILLE SAINT MARC LE BLANC
erreur (adresse non trouvée) pour :5612 L EVANIERE SAINTE MARIE
100 sur 200
erreur (adresse non trouvée) pour :14 COAT PLEN COAT SAINT GOAZEC
150 sur 200
On fait le 11ème sur 21
0 sur 200
erreur (adresse non trouvée) pour :57 COAVOU VILDE GUINGALAN
50 sur 200
100 sur 200
erreur (adresse non trouvée) pour :37 LE COQ ROUGE SERVON SUR VILAINE
erreur (data liste) pour :43 RUE PARC GOVEL GOUDELIN
150 sur 200
erreur (data liste) pour :13 LA PERRIERE MERNEL
erreur (data liste) pour :6 RUE DE LA CROIX DES FORGES NOYAL SUR VILAINE
On fait le 12ème sur 21
0 sur 200
erreur (data liste) pour :64 RUE GEN DE GAULLE GRANDCHAMP
50 sur 200
erreur (data liste) pour :57B RUE ST MICHEL REDON
erreur (data liste) pour :8 RUE BROSSAY ST MARC RENNES
100 sur 200
150 sur 200
erreur (adress

In [ ]:
def recup_pickle(list_df):
    new_df = pd.DataFrame({'No DESIGNATION': [], 'Reussite': [],"ADRESSE":[],"Latitude":[],"Longitude":[]})
    for i in range(0,len(liste)) :
        name = "pickle/gps/df"+str(i+1) +"_lat_long.pkl"
        df_i = pd.read_pickle(name)
        new_df = pandas.concat([new_df, df_i], ignore_index = True)
    return new_df

In [ ]:
df2 = recup_pickle(dfs)

In [ ]:
def ou_sont_les_erreurs(dataframe):
    liste_erreurs = []
    for i in range(0,len(dataframe)):
        if dataframe.loc[i,"Latitude"] == None or dataframe.loc[i,"Longitude"] == None :
            liste_erreurs.append(i)
    return liste_erreurs


In [ ]:
ou_sont_les_erreurs(df2)